In [1]:
# ==========================================
# 0. ライブラリのインストールとインポート
# ==========================================
!pip install datasets tqdm tokenizers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import LambdaLR
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
import tqdm
import math
import time
import gc
import os

In [3]:
# デバイス設定 (GPU推奨)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cpu':
    print("⚠️ 注意: GPUが検出されませんでした。学習は非常に遅くなります。ランタイムの設定を確認してください。")

Using device: cuda


In [4]:
# ==========================================
# 1. データセット作成 (Wiki抽出)
# ==========================================
CORPUS_FILE = "wiki_ja_subset.txt"

if not os.path.exists(CORPUS_FILE):
    print("データセットをダウンロード中...")
    dataset = load_dataset("izumi-lab/wikipedia-ja-20230720", split="train", streaming=True)

    print(f"保存を開始します: {CORPUS_FILE}")
    with open(CORPUS_FILE, "w", encoding="utf-8") as f:
        count = 0
        for data in tqdm.tqdm(dataset):
            text = data["text"].replace("\n", "")
            if len(text) > 100: # 短すぎる記事は除外
                f.write(text + "\n")
                count += 1
            if count >= 10000: # 1万件でストップ
                break
    print("✅ データセット作成完了！")
else:
    print(f"✅ データセットは既に存在します: {CORPUS_FILE}")

データセットをダウンロード中...


README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

保存を開始します: wiki_ja_subset.txt


11724it [00:09, 1264.92it/s]

✅ データセット作成完了！


In [5]:
# ==========================================
# 2. トークナイザの学習 (BPE)
# ==========================================
TOKENIZER_FILE = "custom_tokenizer.json"
VOCAB_SIZE = 32000

# トークナイザの構築
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=False)
tokenizer.decoder = ByteLevelDecoder()

trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["<|endoftext|>", "<|pad|>"],
    show_progress=True
)

print("トークナイザを学習中...")
tokenizer.train([CORPUS_FILE], trainer)
tokenizer.save(TOKENIZER_FILE)
pad_token_id = tokenizer.token_to_id("<|pad|>")
print(f"トークナイザ学習完了。Pad ID: {pad_token_id}")

トークナイザを学習中...
トークナイザ学習完了。Pad ID: 1


In [6]:
# ==========================================
# 3. データセットクラス (省メモリ版)
# ==========================================
class LazyLLMPretrainDataset(Dataset):
    def __init__(self, txt_file, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.txt_file = txt_file
        self.offsets = []

        print("インデックスを作成中 (ファイル位置の記録)...")
        with open(txt_file, "r", encoding="utf-8") as f:
            offset = 0
            for line in f:
                self.offsets.append(offset)
                offset += len(line.encode('utf-8')) # バイト数でオフセットを進める
        print(f"インデックス作成完了: {len(self.offsets)} 行")

    def __len__(self):
        return len(self.offsets)

    def __getitem__(self, idx):
        # 必要な行だけをディスクから読み込む (RAM節約)
        with open(self.txt_file, "r", encoding="utf-8") as f:
            f.seek(self.offsets[idx])
            line = f.readline()

        # トークナイズ
        encoded = self.tokenizer.encode(line)
        ids = encoded.ids

        # 長さ調整 (Truncate or Pad)
        if len(ids) > self.max_length:
            ids = ids[:self.max_length]
        else:
            # 短い場合はパディング (<|pad|> IDを使用)
            ids = ids + [pad_token_id] * (self.max_length - len(ids))

        x = torch.tensor(ids, dtype=torch.long)
        y = torch.tensor(ids, dtype=torch.long)

        return x, y

In [7]:
# ==========================================
# 4. モデル定義 (Llama Architecture)
# ==========================================
from dataclasses import dataclass

@dataclass
class ModelArgs:
    dim: int = 512          # 埋め込み次元
    n_layers: int = 6       # レイヤー数 (8->6 に軽量化)
    n_heads: int = 8        # ヘッド数
    vocab_size: int = 32000
    multiple_of: int = 256
    max_seq_len: int = 256  # コンテキスト長 (ここがメモリに効く)
    dropout: float = 0.1

    @property
    def head_dim(self):
        return self.dim // self.n_heads

class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        return self._norm(x.float()).type_as(x) * self.weight

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)
    freqs = torch.outer(t, freqs).float()
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_cis

def apply_rotary_emb(xq, xk, freqs_cis):
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = freqs_cis[:xq.shape[1]].view(1, xq.shape[1], 1, -1)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

class Attention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads
        self.head_dim = args.head_dim
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

    def forward(self, x, freqs_cis):
        bsz, seqlen, _ = x.shape
        xq = self.wq(x).view(bsz, seqlen, self.n_heads, self.head_dim)
        xk = self.wk(x).view(bsz, seqlen, self.n_heads, self.head_dim)
        xv = self.wv(x).view(bsz, seqlen, self.n_heads, self.head_dim)
        xq, xk = apply_rotary_emb(xq, xk, freqs_cis)

        output = F.scaled_dot_product_attention(
            xq.transpose(1, 2), xk.transpose(1, 2), xv.transpose(1, 2), is_causal=True
        )
        return self.wo(output.transpose(1, 2).contiguous().view(bsz, seqlen, -1))

class FeedForward(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        hidden_dim = 4 * args.dim
        hidden_dim = int(2 * hidden_dim / 3)
        hidden_dim = args.multiple_of * ((hidden_dim + args.multiple_of - 1) // args.multiple_of)
        self.w1 = nn.Linear(args.dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, args.dim, bias=False)
        self.w3 = nn.Linear(args.dim, hidden_dim, bias=False)

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

class TransformerBlock(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.attention_norm = RMSNorm(args.dim)
        self.attention = Attention(args)
        self.ffn_norm = RMSNorm(args.dim)
        self.feed_forward = FeedForward(args)

    def forward(self, x, freqs_cis):
        h = x + self.attention(self.attention_norm(x), freqs_cis)
        out = h + self.feed_forward(self.ffn_norm(h))
        return out

class Transformer(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.args = args
        self.tok_embeddings = nn.Embedding(args.vocab_size, args.dim)
        self.layers = nn.ModuleList([TransformerBlock(args) for _ in range(args.n_layers)])
        self.norm = RMSNorm(args.dim)
        self.output = nn.Linear(args.dim, args.vocab_size, bias=False)
        self.freqs_cis = precompute_freqs_cis(self.args.dim // self.args.n_heads, self.args.max_seq_len * 2)

    def forward(self, idx):
        bsz, seqlen = idx.shape
        x = self.tok_embeddings(idx)
        freqs_cis = self.freqs_cis[:seqlen].to(x.device)
        for layer in self.layers:
            x = layer(x, freqs_cis)
        x = self.norm(x)
        return self.output(x)

In [8]:
# ==========================================
# 5. 学習設定 & ループ (最適化済み)
# ==========================================
# 軽量設定
BATCH_SIZE = 8           # VRAM節約のため小さく
ACCUMULATION_STEPS = 4   # 8 * 4 = 32 (実質バッチサイズ)
EPOCHS = 1               # デモのため1エポック
MAX_LENGTH = 256

# データセット & ローダー
dataset = LazyLLMPretrainDataset(CORPUS_FILE, tokenizer, max_length=MAX_LENGTH)
# num_workers=0 が重要 (ColabのRAMクラッシュ防止)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# モデル初期化
args = ModelArgs(
    dim=512, n_layers=6, n_heads=8,
    vocab_size=VOCAB_SIZE, max_seq_len=MAX_LENGTH
)
model = Transformer(args).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

# Optimizer & Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), weight_decay=0.1)
total_steps = len(dataloader) * EPOCHS // ACCUMULATION_STEPS

def get_scheduler(optimizer, warmup_steps, training_steps):
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return float(current_step) / float(max(1, warmup_steps))
        progress = float(current_step - warmup_steps) / float(max(1, training_steps - warmup_steps))
        return max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
    return LambdaLR(optimizer, lr_lambda)

scheduler = get_scheduler(optimizer, int(total_steps * 0.1), total_steps)
criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id) # パディングはLoss計算から除外
scaler = GradScaler() # Mixed Precision用

print("\n🚀 学習を開始します...")
model.train()
optimizer.zero_grad()

start_time = time.time()
total_loss = 0

for step, (x, y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)

    # 1. Forward (Mixed Precision)
    with autocast(device_type="cuda", dtype=torch.float16): # T4ならfloat16推奨
        logits = model(x)

        # Shift for prediction
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = y[..., 1:].contiguous()

        loss = criterion(shift_logits.view(-1, args.vocab_size), shift_labels.view(-1))

        # 2. 勾配蓄積のためのLoss割り算
        loss = loss / ACCUMULATION_STEPS

    # 3. Backward
    scaler.scale(loss).backward()

    # 4. Update (一定ステップごとに実行)
    if (step + 1) % ACCUMULATION_STEPS == 0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        optimizer.zero_grad(set_to_none=True) # メモリ解放

    total_loss += loss.item() * ACCUMULATION_STEPS

    # ログ表示
    if step % 100 == 0:
        elapsed = time.time() - start_time
        current_lr = scheduler.get_last_lr()[0]
        print(f"Step {step}/{len(dataloader)} | Loss: {loss.item() * ACCUMULATION_STEPS:.4f} | LR: {current_lr:.6f} | Time: {elapsed:.1f}s")

        # メモリ掃除
        gc.collect()
        torch.cuda.empty_cache()

print("\n🎉 学習完了！")

インデックスを作成中 (ファイル位置の記録)...
インデックス作成完了: 10000 行
Model parameters: 53.2M

🚀 学習を開始します...
Step 0/1250 | Loss: 10.5534 | LR: 0.000000 | Time: 0.9s
Step 100/1250 | Loss: 9.2830 | LR: 0.000242 | Time: 13.7s
Step 200/1250 | Loss: 8.5183 | LR: 0.000297 | Time: 25.6s
Step 300/1250 | Loss: 8.0674 | LR: 0.000282 | Time: 37.4s
Step 400/1250 | Loss: 7.9703 | LR: 0.000258 | Time: 49.8s
Step 500/1250 | Loss: 8.0296 | LR: 0.000225 | Time: 62.0s
Step 600/1250 | Loss: 7.7718 | LR: 0.000186 | Time: 74.4s
Step 700/1250 | Loss: 7.7683 | LR: 0.000144 | Time: 85.9s
Step 800/1250 | Loss: 7.6845 | LR: 0.000103 | Time: 98.3s
Step 900/1250 | Loss: 7.7206 | LR: 0.000066 | Time: 110.5s
Step 1000/1250 | Loss: 7.5657 | LR: 0.000035 | Time: 122.8s
Step 1100/1250 | Loss: 7.6166 | LR: 0.000030 | Time: 135.3s
Step 1200/1250 | Loss: 7.3080 | LR: 0.000030 | Time: 147.7s

🎉 学習完了！


In [9]:
# ==========================================
# 6. 生成テスト
# ==========================================
def generate_text(model, start_text, max_new_tokens=50):
    model.eval()
    ids = tokenizer.encode(start_text).ids
    input_ids = torch.tensor([ids], device=device)

    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_ids)
            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            input_ids = torch.cat([input_ids, next_token], dim=1)

            # 終了タグが出たら止める
            if next_token.item() == tokenizer.token_to_id("<|endoftext|>"):
                break

    return tokenizer.decode(input_ids[0].tolist())

print("\n--- 生成結果 ---")
print(generate_text(model, "人工知能は"))


--- 生成結果 ---
人工知能は、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本の小説家、日本
